In [26]:
from time import time

begin = time()
print(f"Importing torch ...")
import torch
import torch.nn as nn
print(f"torch imported in {time()-begin:.2f} seconds")

Importing torch ...
torch imported in 0.00 seconds


In [29]:
import torch
import torch.nn as nn

def grad_h(u):
    # Compute the gradient in both x and y directions
    grad_u = torch.zeros(u.size() + (2,), device=u.device)
    grad_u[..., 0] = torch.diff(u, n=1, dim=0, append=u[-1:, ...])  # Gradient in the x direction
    grad_u[..., 1] = torch.diff(u, n=1, dim=1, append=u[:, -1:])    # Gradient in the y direction
    return grad_u

def div_h(v):
    # Compute the divergence from the gradient in both x and y directions
    div_v = torch.zeros(v.size()[:-1], device=v.device)
    div_v += torch.diff(v[..., 0], n=1, dim=0, prepend=v[0:1, ..., 0])  # Divergence in the x direction
    div_v += torch.diff(v[..., 1], n=1, dim=1, prepend=v[:, 0:1, ..., 1])  # Divergence in the y direction
    return div_v

class CustomAlgorithm(nn.Module):
    def __init__(self):
        super(CustomAlgorithm, self).__init__()
        self.sigma = nn.Parameter(torch.tensor(1.0), requires_grad=True)
        self.tau = nn.Parameter(torch.tensor(1.0), requires_grad=True)
        self.theta = nn.Parameter(torch.tensor(1.0), requires_grad=True)

        with torch.no_grad():
            while self.sigma * self.tau * 0.5 * (17 + torch.sqrt(torch.tensor(33.0))) > 1:
                self.sigma /= 2
                self.tau /= 2

    def forward(self, u0, p0, num_iters=100):
        """
        
        Parameters
        ----------
        u0 : torch.Tensor
            Initial guess for the solution u. 
            Expect 2D tensor of shape [n, n] (for now).
        """
        device = u0.device
        u0 = u0.to(device)
        p0 = p0.to(device)

        v0 = torch.zeros_like(grad_h(u0)).to(device)  # Adjusted to match gradient size
        w0 = torch.zeros_like(grad_h(p0)).to(device)  # Adjusted to match gradient size
        u_bar0 = u0.clone().detach().to(device) # shape = [10, 10]
        p_bar0 = p0.clone().detach().to(device) # shape = [10, 10]

        for i in range(num_iters):
            if i == 0:
                print(f"grad_h(u_bar0).shape = {grad_h(u_bar0).shape}")
                print(f"p_bar0.shape = {p_bar0.shape}")
            vn_plus1 = self.P_alpha1(v0 + self.sigma * (grad_h(u_bar0) - p_bar0))
            wn_plus1 = self.P_alpha0(w0 + self.sigma * grad_h(p0))
            un_plus1 = self.id_tauFh_inverse(u0 + self.tau * div_h(vn_plus1))
            pn_plus1 = p0 + self.tau * (vn_plus1 + div_h(wn_plus1))
            u_bar0 = 2 * un_plus1 - u0
            p_bar0 = 2 * pn_plus1 - p0

            u0, p0 = un_plus1, pn_plus1
            v0, w0 = vn_plus1, wn_plus1

        return u0

    def P_alpha1(self, x):
        # Placeholder for projection operator P_alpha1
        return x

    def P_alpha0(self, x):
        # Placeholder for projection operator P_alpha0
        return x

    def id_tauFh_inverse(self, x):
        # Placeholder for (id + τ∂Fh)^(-1)
        return x



In [30]:
# Example usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
u0 = torch.zeros((10, 10), device=device)
p0 = torch.zeros((10, 10), device=device)
model = CustomAlgorithm().to(device)

uN = model(u0, p0, N=100)
print(uN)


grad_h(u_bar0).shape = torch.Size([10, 10, 2])
p_bar0.shape = torch.Size([10, 10])


RuntimeError: The size of tensor a (2) must match the size of tensor b (10) at non-singleton dimension 2

In [ ]:
import numpy as np

# Assuming some necessary functions and projections are defined
def P_alpha1(x):
    # Placeholder for projection operator P_alpha1
    return x

def P_alpha0(x):
    # Placeholder for projection operator P_alpha0
    return x

def id_tauFh_inverse(x):
    # Placeholder for (id + τ∂Fh)^(-1)
    return x

def Fh(u):
    # Placeholder for the function Fh
    return u

def TGV2_alpha(u):
    # Placeholder for the TGV^2_α function
    return u

def grad_h(u):
    # Placeholder for the gradient operator ∇h
    return np.gradient(u)

def div_h(v):
    # Placeholder for the divergence operator div_h
    return np.divergence(v)

# Parameters
sigma = 1.0
tau = 1.0
while sigma * tau * 0.5 * (17 + np.sqrt(33)) > 1:
    sigma /= 2
    tau /= 2

# Initial variables
u0, p0 = np.zeros((10, 10)), np.zeros((10, 10))
v0, w0 = np.zeros((10, 10)), np.zeros((10, 10))
u_bar0, p_bar0 = u0.copy(), p0.copy()

# Number of iterations
num_iters = 100

# Iteration loop
for i in range(num_iters):
    vn_plus1 = P_alpha1(v0 + sigma * (grad_h(u_bar0) - p_bar0))
    wn_plus1 = P_alpha0(w0 + sigma * grad_h(p0))
    un_plus1 = id_tauFh_inverse(u0 + tau * div_h(vn_plus1))
    pn_plus1 = p0 + tau * (vn_plus1 + div_h(wn_plus1))
    u_bar0 = 2 * un_plus1 - u0
    p_bar0 = 2 * pn_plus1 - p0

    # Update for next iteration
    u0, p0 = un_plus1, pn_plus1
    v0, w0 = vn_plus1, wn_plus1

# Result
uN = u0

# Output the result
print(uN)


TypeError: can't multiply sequence by non-int of type 'float'